# 📌 Attach Default Lakehouse
❗**Note the code in the cell that follows is required to programatically attach the lakehouse and enable the running of spark.sql(). If this cell fails simply restart your session as this cell MUST be the first command executed on session start.**

In [ ]:
%%configure
{
    "defaultLakehouse": {  
        "name": "{{lakehouse_name}}",
    }
}

# 📦 Pip
Pip installs reqired specifically for this template should occur here

In [ ]:
# No pip installs needed for this notebook

# 🔗 Imports

In [ ]:
from notebookutils import mssparkutils # type: ignore

# #️⃣ Functions

In [ ]:
def remove_file_using_notebookutils(file):
    """Remove a file using notebookutils."""
    try:
        mssparkutils.fs.rm(file, True)
    except:
        pass


def create_path_using_notebookutils(path):
    """Create a path using notebookutils."""
    mssparkutils.fs.mkdirs(path)

# Prepare

In [ ]:
log_location = 'Files/Control/'
create_path_using_notebookutils(log_location)
remove_file_using_notebookutils(log_location+'dbt_execution.log')


# Executions for Each Run Order Below:
